In [65]:
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset
import soundfile as sf

In [66]:
model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

In [223]:
from pydub import AudioSegment
track = AudioSegment.from_file("samples/lefan-sample.m4a",  format= 'm4a')

# Set the target parameters
sample_rate = 16000
channels = 1
sample_width = 2

track = track.set_frame_rate(sample_rate).set_channels(channels).set_sample_width(sample_width)
file_handle = track.export("test.wav", format='wav')

In [254]:
speech, _ = sf.read("samples/audio4.wav")

In [255]:
speech

array([-0.00024414, -0.00048828, -0.00048828, ...,  0.00061035,
        0.00012207,  0.00036621])

In [256]:
inputs = processor(speech, sampling_rate=16000, return_tensors="pt")
# generated_ids = model.generate(input_ids=inputs["input_features"], attention_mask=inputs["attention_mask"])
generated_ids = model.generate(input_features=inputs["input_features"], attention_mask=inputs["attention_mask"])

In [257]:
transcription = processor.batch_decode(generated_ids)

In [258]:
transcription

['</s> microscopically there was inthysima fibrosis and vascular congestion</s>']